# jQueryUI tests

These tests verify that JQueryUI loads with CSS styling when requested.

In [1]:
from jp_proxy_widget import notebook_test_helpers

validators = notebook_test_helpers.ValidationSuite()

In [2]:
import jp_proxy_widget
from jp_proxy_widget import js_context

class JQueryUIDialogTestWidget(jp_proxy_widget.JSProxyWidget):
    
    "A jQueryUI dialog with custom styling that reports back to Python when it is closed."
    
    def __init__(self, *pargs, **kwargs):
        super(JQueryUIDialogTestWidget, self).__init__(*pargs, **kwargs)
        self.is_open = "unknown"
        self.check_jquery()
        random_style = """
        .dialog-random-style-for-testing2 {
            color: rgb(216, 50, 61);
            background-color: rgb(200, 218, 61);
        }
        """
        self.load_css_text("dialog-random-stylesheet", random_style)

        self.js_init("""
            // Add dialog as a child to the widget element.
            element.empty();
            var dialog = $('<div><b class="dialog-random-style-for-testing2">Hello from jQueryUI</b></div>')
            .appendTo(element);

            // Add some callables to the element accessible from Python.
            element.report_status = function() {
                // Report whether the dialog is now open back to Python.
                is_open_callback(dialog.dialog("isOpen"))
            };

            element.close = function () {
                // Close the dialog.
                dialog.dialog("close");
            };
            
            // Open the dialog and have it report its open status when it's closed.
            dialog.dialog({close: element.report_status});
            
            element.report_status();
        """, is_open_callback=self.is_open_callback)
        
    def is_open_callback(self, value):
        "Python callback which remembers whether the dialog reported itself open or not."
        self.is_open = value
            


In [3]:
# Create and display the dialog.
test_dialog = JQueryUIDialogTestWidget()
test_dialog

JQueryUIDialogTestWidget(status='Not yet rendered')

In [4]:
# NOTE: Click here to focus on the cell (otherwise "enter" with the dialog focused will close the dialog).
test_dialog.is_open

'unknown'

In [5]:
# Close the dialog by calling the element.close() method.
test_dialog.element.close()

element

In [6]:
def validate_test_dialog():
    assert test_dialog.is_open == False
    print('test_dialog apparently ok!')

validators.add_validation(test_dialog, validate_test_dialog)

In [7]:
test_slider_css = """
  #test-slider-custom-handle {
    width: 3em;
    height: 1.6em;
    top: 50%;
    margin-top: -.8em;
    text-align: center;
    line-height: 1.6em;
  }
  #test-slider-container {
    width: 50em;
    height: 10em;
  }
"""

class JQueryUISliderTestWidget(jp_proxy_widget.JSProxyWidget):
    
    "A slider adapted from http://jqueryui.com/slider/#custom-handle"
    
    def __init__(self, *pargs, **kwargs):
        super(JQueryUISliderTestWidget, self).__init__(*pargs, **kwargs)
        self.slider_value = "unknown"
        
        self.load_css_text("test-slider-stylesheet", test_slider_css)

        self.js_init("""
            element.html('<div class="test-slider-container"></div>');
            var slider_div = $("<div></div>").appendTo(element);
            var handle = $('<div class="test-slider-custom-handle ui-slider-handle"></div>')
            .width("30px")
            .appendTo(slider_div);
            slider_div.slider({
              min: -50,
              max: 65,
              value: 43,
              create: function() {
                //handle.text( element.slider( "value" ) ); // causes error
              },
              slide: function( event, ui ) {
                handle.text( ui.value );
              },
              change: function( event, ui ) {
                handle.text( ui.value );
                report_change( ui.value );
              }
            });
            element.height("50px").width("300px").css("background-color", "cornsilk");
            slider_div.width("250px");
            slider_div.position({
              my: "center",
              at: "center",
              of: element
            });
            element.set_value = function(value) {
                slider_div.slider("value", value);
            };
            element.set_value(44);
        """, report_change=self.report_change)
        
    def report_change(self, value):
        self.slider_value = value
        

In [8]:
slider_widget = JQueryUISliderTestWidget()
slider_widget

JQueryUISliderTestWidget(status='Not yet rendered')

In [9]:
slider_widget.slider_value
#slider_widget.print_status()

'unknown'

In [10]:
slider_widget.element.set_value(-15)

def validate_slider_widget():
    assert slider_widget.slider_value == -15
    print('slider_widget apparently ok!')

validators.add_validation(slider_widget, validate_slider_widget)

In [11]:
# This should be the last cell
validators.run_all_in_widget()

sleeping in kernel interface...
initializing validator widget.
